$\newcommand{\xv}{\mathbf{x}}
\newcommand{\Xv}{\mathbf{X}}
\newcommand{\yv}{\mathbf{y}}
\newcommand{\zv}{\mathbf{z}}
\newcommand{\av}{\mathbf{a}}
\newcommand{\Wv}{\mathbf{W}}
\newcommand{\wv}{\mathbf{w}}
\newcommand{\tv}{\mathbf{t}}
\newcommand{\Tv}{\mathbf{T}}
\newcommand{\muv}{\boldsymbol{\mu}}
\newcommand{\sigmav}{\boldsymbol{\sigma}}
\newcommand{\phiv}{\boldsymbol{\phi}}
\newcommand{\Phiv}{\boldsymbol{\Phi}}
\newcommand{\Sigmav}{\boldsymbol{\Sigma}}
\newcommand{\Lambdav}{\boldsymbol{\Lambda}}
\newcommand{\half}{\frac{1}{2}}
\newcommand{\argmax}[1]{\underset{#1}{\operatorname{argmax}}}
\newcommand{\argmin}[1]{\underset{#1}{\operatorname{argmin}}}$

# Training Multiple Models to Obtain Confidence Intervals

Topics for today:
1. Linear model code from last time
2. Irregularly Spaced Data
1. Divide data into training and testing sets
1. Multiple Models to Estimate Uncertainties and Confidence Intervals
1. 90% confidence interval for predictions of all samples
1. Confidence intervals of the weights

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

## Linear model code from last time

In [ ]:
from topic_banner import new_topic
new_topic('Linear model code from last time')

In [ ]:
def make_powers(X, max_power):
    return np.hstack([X ** p for p in range(1, max_power + 1)])

In [ ]:
def train(X, T, n_epochs, rho):
    
    means = X.mean(0)
    stds = X.std(0)
    # Replace stds of 0 with 1 to avoid dividing by 0.
    stds[stds == 0] = 1
    Xst = (X - means) / stds
    
    Xst = np.insert(Xst, 0, 1, axis=1)  # Insert column of 1's as first column in Xst
    
    # n_samples, n_inputs = Xst.shape[0]
    n_samples, n_inputs = Xst.shape
    
    # Initialize weights to all zeros
    W = np.zeros((n_inputs, 1))  # matrix of one column
    
    # Repeat updates for all samples for multiple passes, or epochs,
    for epoch in range(n_epochs):
        
        # Update weights once for each sample.
        for n in range(n_samples):
        
            # Calculate prediction using current model, w.
            #    n:n+1 is used instead of n to preserve the 2-dimensional matrix structure
            Y = Xst[n:n + 1, :] @ W
            
            # Update w using negative gradient of error for nth sample
            W += rho * Xst[n:n + 1, :].T * (T[n:n + 1, :] - Y)
                
    # Return a dictionary containing the weight matrix and standardization parameters.
    return {'W': W, 'means' : means, 'stds' :stds, 'max_power': max_power}

def use(model, X):
    Xst = (X - model['means']) / model['stds']
    Xst = np.insert(Xst, 0, 1, axis=1)
    Y = Xst @ model['W']
    return Y

def rmse(A, B):
    return np.sqrt(np.mean( (A - B)**2 ))

## Irregularly Spaced Data

In [ ]:
from topic_banner import new_topic
new_topic('Irregularly Spaced Data')

Here is a function of a single variable, $x$, which we will apply to three different spans of $x$ values.

$$ -1 + 10 e^{0.1 x} + 0.1 x^2 - 0.02 x^3 + r$$

where $r$ is from a standard Normal distribution.

In [ ]:
n_samples_each_section = 40

ns = n_samples_each_section
X = np.hstack((np.linspace(-8, -5, num=ns),
               np.linspace(0, 3, num=ns),
               np.linspace(6, 10, num=ns))).reshape(3 * ns, 1)
T = -1 + 10 * np.exp(0.1 * X)
T += 0.1 * X**2
T += - 0.02 * X**3
T += 1.0 * np.random.normal(size=(3 * ns, 1))
X.shape, T.shape

In [ ]:
plt.plot(X, T, '.-');

## Divide data into training and testing sets

In [ ]:
new_topic('Divide data into training and testing sets')

In [ ]:
round(7.8)

In [ ]:
training_fraction = 0.8

n_rows = X.shape[0]
row_indices = np.arange(n_rows)
np.random.shuffle(row_indices)
n_train = round(n_rows * training_fraction)
n_test = n_rows - n_train

Xtrain = X[row_indices[:n_train], :]
Ttrain = T[row_indices[:n_train], :]
Xtest = X[row_indices[n_train:], :]
Ttest = T[row_indices[n_train:], :]

Xtrain.shape, Ttrain.shape, Xtest.shape, Ttest.shape

In [ ]:
plt.plot(Xtrain[:, 0], Ttrain, 'o', label='Train')
plt.plot(Xtest[:, 0], Ttest, 'ro', label='Test')
plt.legend(loc='best');

## Multiple Models to Estimate Uncertainties and Confidence Intervals

In [ ]:
new_topic('Multiple Models to Estimate Uncertainties and Confidence Intervals')

Make models based on bootstrap samples of training data.  `models` will be list of models, one for each bootstrap sample.

For each bootstrap sample of our training data we will randomly choose `n_train` samples **with replacement**.  The following code cell illustrates how to create 20 bootstrap samples, each with 10 samples.  The bootstrap samples are defined as row indices.

In [ ]:
np.random.choice(list(range(11)), 20)

In [ ]:
max_power = 1
n_models = 1000
max_power = 1  # linear model

Xtrain = X[row_indices[:n_train], :]
Xtest = X[row_indices[n_train:], :]
Xtrain = make_powers(Xtrain, max_power)
Xtest = make_powers(Xtest, max_power)

n_epochs = 1000
rho = 0.01

n_models = 10

models = []
for model_i in range(n_models):
    train_rows = np.random.choice(list(range(n_train)), n_train)
    Xtrain_boot = Xtrain[train_rows, :]
    Ttrain_boot = Ttrain[train_rows, :]
    model = train(Xtrain_boot, Ttrain_boot, n_epochs, rho)
    models.append(model)
    print(f'Model {model_i}', end=' ')

In [ ]:
len(models)

In [ ]:
models[0]

Now we will apply all of the models to the test data.

In [ ]:
use(models[0], Xtest)

In [ ]:
Y_all = []
for model in models:
    Y_all.append( use(model, Xtest) )

In [ ]:
len(Y_all)

In [ ]:
Y_all[0].shape

Let's create a `numpy.array` for all outputs of all models so we can easily calculate the mean for each test sample over all models.

In [ ]:
np.array(Y_all).shape

Use `numpy.squeeze` to wring out the "unused" dimension.

In [ ]:
np.array(Y_all).squeeze().shape

In [ ]:
Y_all = np.array(Y_all).squeeze().T  # I like putting each model's output in a column, so `Y_all` now has each model's output for a sample in a row.
Ytest = np.mean(Y_all, axis=1)

In [ ]:
Ytest.shape

In [ ]:
RMSE_test = np.sqrt(np.mean((Ytest - Ttest)**2))
print(f'Test RMSE is {RMSE_test:.4f}')

In [ ]:
n_plot = 200
Xplot = np.linspace(-10, 12, n_plot).reshape(n_plot, 1)
Xplot_powers = make_powers(Xplot, max_power)
Ys = []
for model in models:
    Yplot = use(model, Xplot_powers)
    Ys.append(Yplot)

In [ ]:
len(Ys)

In [ ]:
Ys[0].shape

In [ ]:
np.array(Ys).shape

In [ ]:
Ys = np.array(Ys).squeeze().T
Ys.shape

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(Xtrain[:, 0], Ttrain, 'o')
plt.plot(Xtest[:, 0], Ttest, 'o')
plt.plot(Xplot, Ys, alpha=0.5);

Do again with nonlinear terms.

In [ ]:
max_power = 8
Xtrain = X[row_indices[:n_train], :]
Xtest = X[row_indices[n_train:], :]
Xtrain = make_powers(Xtrain, max_power)
Xtest = make_powers(Xtest, max_power)

n_epochs = 2000
rho = 0.02

n_models = 100 

models = []
for model_i in range(n_models):
    train_rows = np.random.choice(list(range(n_train)), n_train)
    Xtrain_boot = Xtrain[train_rows, :]
    Ttrain_boot = Ttrain[train_rows, :]
    model = train(Xtrain_boot, Ttrain_boot, n_epochs, rho)
    models.append(model)
    print(f'Model {model_i}', end=' ')

In [ ]:
n_plot = 200
Xplot = np.linspace(-10, 12, n_plot).reshape(n_plot, 1)
Xplot_powers = make_powers(Xplot, max_power)
Ys = []
for model in models:
    Yplot = use(model, Xplot_powers)
    Ys.append(Yplot)

Ys = np.array(Ys).squeeze().T

plt.figure(figsize=(10, 10))
plt.plot(Xtrain[:, 0], Ttrain, 'o')
plt.plot(Xtest[:, 0], Ttest, 'o')
plt.plot(Xplot, Ys, alpha=0.5);

## 90% confidence interval for predictions of all samples.

In [ ]:
new_topic('90% confidence interval for predictions of all samples')

In [ ]:
n_plot = 200
Xplot = np.linspace(-10, 12, n_plot).reshape(n_plot, 1)
Xplot_powers = make_powers(Xplot, max_power)
Ys = []
for model in models:
    Yplot = use(model, Xplot_powers)
    Ys.append(Yplot)

Ys = np.array(Ys).squeeze().T

plt.figure(figsize=(10, 10))
plt.plot(Xtrain[:, 0], Ttrain, 'o', alpha=0.2)
plt.plot(Xtest[:, 0], Ttest, 'o', alpha=0.2)

plt.fill_between(Xplot.reshape(-1), Ys.min(axis=-1), Ys.max(axis=-1),
                color='#fded08')

middle = len(models) // 2
plt.plot(Xplot, Ys[:, middle]);

## Confidence intervals of the weights

In [ ]:
new_topic('Confidence intervals of the weights')

Now to evaluate the significance of each input by considering the weights on each input across the models. Let's say we want the 90% confidence interval. First, let's collect the weights.

In [ ]:
all_Ws = [model['W'] for model in models]
len(all_Ws), all_Ws[0].shape

In [ ]:
np.array(all_Ws).shape

In [ ]:
all_Ws = np.array(all_Ws).squeeze()
all_Ws.shape

Now we must sort the weight values independently for each input to find the 5% and 95% quantile values.

In [ ]:
Z = np.random.randint(-10, 10, size=50).reshape(10, 5)
Z

In [ ]:
np.sort(Z)

In [ ]:
Z

In [ ]:
np.sort(Z, axis=0)

There we go.

In [ ]:
all_Ws = np.sort(all_Ws, axis=0)
low_high = all_Ws[[4, 94], :].T
low_high

In [ ]:
for i, row in enumerate(low_high):
    if i == 0:
        print(f'Bias w   Low {row[0]:6.2f} High {row[1]:6.2f}')
    else:
        print(f'Power {i + 1:2} Low {row[0]:6.2f} High {row[1]:6.2f}')